In [1]:
import tensorflow as tf
tf.__version__

'2.4.0'

In [2]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
import handshape_datasets as hd
from IPython import display
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, LeakyReLU, Dropout, Flatten
from tensorflow.keras import Model

### Check GPU

In [ ]:
#Check GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
print(tf.config.experimental.list_logical_devices('GPU'))
tf.test.is_gpu_available()

## Load dataset

In [162]:
data = hd.load('lsa16')

good_min = 40
good_classes = []
n_unique = len(np.unique(data[1]['y']))
for i in range(n_unique):
    images = data[0][np.equal(i, data[1]['y'])]
    if len(images) >= good_min:
        good_classes = good_classes + [i]
        
x = data[0][np.in1d(data[1]['y'], good_classes)]
img_shape = x[0].shape

y = data[1]['y'][np.in1d(data[1]['y'], good_classes)]
y_dict = dict(zip(np.unique(y), range(len(np.unique(y)))))
y = np.vectorize(y_dict.get)(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2, stratify=y)

classes = np.unique(y_train)
n_classes = len(classes)

INFO:Loading lsa16...
INFO:Loading default version: color


## Metrics

In [163]:
from tensorflow.keras import backend as K
# Definición de las métricas F1, recall y precision utilizando Keras.

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))


## Model

In [164]:
def model():
    model = tf.keras.Sequential()
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same',input_shape=[32, 32, 3]))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(n_classes, activation='softmax'))

    return model

## Train

In [123]:
model = model()
model.summary()
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', f1_m, precision_m, recall_m])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 16, 16, 64)        4864      
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 8, 8, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)             

In [124]:
with tf.device('/CPU:0'):
    model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
5/5 [==============================] - 1s 117ms/step - loss: 22.7238 - accuracy: 0.1604 - f1_m: 1.2385 - precision_m: 1.1974 - recall_m: 1.2832 - val_loss: 4.9372 - val_accuracy: 0.4750 - val_f1_m: 1.3466 - val_precision_m: 1.3285 - val_recall_m: 1.3658
Epoch 2/20
5/5 [==============================] - 0s 93ms/step - loss: 4.6485 - accuracy: 0.5793 - f1_m: 1.1752 - precision_m: 1.1446 - recall_m: 1.2076 - val_loss: 2.6656 - val_accuracy: 0.7500 - val_f1_m: 1.1821 - val_precision_m: 1.1585 - val_recall_m: 1.2067
Epoch 3/20
5/5 [==============================] - 0s 87ms/step - loss: 1.9602 - accuracy: 0.7904 - f1_m: 1.1032 - precision_m: 1.0697 - recall_m: 1.1392 - val_loss: 1.8934 - val_accuracy: 0.8625 - val_f1_m: 1.0738 - val_precision_m: 1.0469 - val_recall_m: 1.1023
Epoch 4/20
5/5 [==============================] - 0s 86ms/step - loss: 0.8402 - accuracy: 0.8824 - f1_m: 1.0767 - precision_m: 1.0459 - recall_m: 1.1096 - val_loss: 1.6569 - val_accuracy: 0.8875 - val_f1_m: 1.

## Classifier with dataset augmentation

In [4]:
data = hd.load('lsa16')

good_min = 40
good_classes = []
n_unique = len(np.unique(data[1]['y']))
for i in range(n_unique):
    images = data[0][np.equal(i, data[1]['y'])]
    if len(images) >= good_min:
        good_classes = good_classes + [i]
        
x = data[0][np.in1d(data[1]['y'], good_classes)]
img_shape = x[0].shape

y = data[1]['y'][np.in1d(data[1]['y'], good_classes)]
y_dict = dict(zip(np.unique(y), range(len(np.unique(y)))))
y = np.vectorize(y_dict.get)(y)

# shapes
img_n , height, weight, chanels = x.shape

#Normalization
x = x.reshape(img_n, height, weight, chanels).astype('float32')
x = (x - 127.5) / 127.5  # Normalize the images to [-1, 1]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2, stratify=y)

classes = np.unique(y_train)
n_classes = len(classes)

#x_tuple = list(zip(x_train_aug,y_train))
#x_train_aug

print(x_train.shape, y_train.shape)

INFO:Loading lsa16...
INFO:Loading default version: color


(640, 32, 32, 3) (640,)


## Generator model

In [7]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(int(height/4)*int(weight/4)*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((int(height/4), int(weight/4), 256)))

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    
    return model

In [8]:
generator = make_generator_model()
generator.summary()
num_examples_to_generate = 15
noise = tf.random.normal([num_examples_to_generate, 100])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16384)             1638400   
_________________________________________________________________
batch_normalization (BatchNo (None, 16384)             65536     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 16384)             0         
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 128)         819200    
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 128)         512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 8, 8, 128)         0

## Assembling the dataset

In [10]:
weights_path = '../GANs/results/models/Basic-GAN/lsa16-generator-class-{}/generator_class_{}_{}.h5'
for i in range(16):
    for j in range(3):
        generator.load_weights(weights_path.format(i,i,j))
        generated_image = generator(noise, training=False).numpy()
        x_train = np.concatenate((x_train, generated_image), axis=0)
        y_tmp = np.ones(num_examples_to_generate,)* i
        y_train = np.concatenate((y_train,y_tmp), axis=0)


In [169]:
print(x_train.shape, y_train.shape)

(1360, 32, 32, 3) (1360,)


In [170]:
shuffler = np.random.permutation(x_train.shape[0])
x_train = x_train[shuffler]
y_train = y_train[shuffler]
print(x_train.shape, y_train.shape)

(1360, 32, 32, 3) (1360,)


In [171]:
model = model()
model.summary()
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', f1_m, precision_m, recall_m])

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 16, 16, 64)        4864      
_________________________________________________________________
leaky_re_lu_39 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
leaky_re_lu_40 (LeakyReLU)   (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 8, 8, 128)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8192)            

In [172]:
with tf.device('/CPU:0'):
    model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
11/11 [==============================] - 1s 99ms/step - loss: 2.3559 - accuracy: 0.2977 - f1_m: 11.4184 - precision_m: 5700729834.6667 - recall_m: 5.8362 - val_loss: 0.7118 - val_accuracy: 0.7688 - val_f1_m: 2.4525 - val_precision_m: 2.6869 - val_recall_m: 2.2558
Epoch 2/20
11/11 [==============================] - 1s 83ms/step - loss: 0.4629 - accuracy: 0.8709 - f1_m: 1.5477 - precision_m: 1.5751 - recall_m: 1.5231 - val_loss: 0.6737 - val_accuracy: 0.8062 - val_f1_m: 1.4950 - val_precision_m: 1.4798 - val_recall_m: 1.5108
Epoch 3/20
11/11 [==============================] - 1s 75ms/step - loss: 0.3246 - accuracy: 0.9094 - f1_m: 1.2420 - precision_m: 1.2144 - recall_m: 1.2710 - val_loss: 0.4246 - val_accuracy: 0.8687 - val_f1_m: 1.3535 - val_precision_m: 1.3476 - val_recall_m: 1.3594
Epoch 4/20
11/11 [==============================] - 1s 73ms/step - loss: 0.1500 - accuracy: 0.9557 - f1_m: 1.2120 - precision_m: 1.1896 - recall_m: 1.2354 - val_loss: 0.3494 - val_accuracy: 0.887